# Text generation with LSTMs
![](https://tc.sinaimg.cn/maxwidth.800/tc.service.weibo.com/cdn_images_1_medium_com/58ad765e09eacb5116c9dfc5897c7296.png)

This notebook is an attempt to reproduce the results from the official Keras example on text generation. <br>
All credits are to François CHOLLET, the Keras author. 

In [1]:
# Usual
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
import time
from tqdm import tqdm

# Autoreload
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
%matplotlib inline

In [2]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

Using TensorFlow backend.


***
# Preparing the dataset
## Getting the raw data

##### Read a dataset of songs lyrics

In [3]:
dataset = pd.read_csv("C:/data/text_generation/songdata.csv")

##### Select a subset of 1000 songs

In [4]:
text = " ".join(list(dataset.text.head(1000)))

## Preprocessing the data

In [5]:
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

corpus length: 1170215
total chars: 76


In [6]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 390059


In [7]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


***
# Create the model

##### Creating the model

In [12]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


##### Helper function

In [13]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [29]:
def generate_song(start_sentence = None,temperature = [0.2,0.5,1.0,1.2]):
    if type(temperature)!=list: temperature = [temperature]
    if start_sentence is None:
        start_index = random.randint(0, len(text) - maxlen - 1)
        start_sentence = text[start_index: start_index + maxlen]
    else:
        augment = lambda x : " "*(40-len(x))+x
        start_sentence = augment(start_sentence)

    for diversity in temperature:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = start_sentence
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [15]:
# train the model, output generated text after each iteration
for iteration in range(1, 10):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x, y,
              batch_size=128,
              epochs=1,verbose = 2)
    generate_song()




--------------------------------------------------
Iteration 1
Epoch 1/1
664s - loss: 1.7153

----- diversity: 0.2
----- Generating with seed: "ot just hear you  
Dumb dumb diddle, to "
ot just hear you  
Dumb dumb diddle, to me  
  
There wasn't stay  
So there wasn't go the same  
I can't leave you  
  
We're gonna stay  
  
I want to be the soun  
  
I can't leave my more  
  
I want to stay the stare  
  
I'm a pieling  
I'm gonna stay  
  
We got a lone  
  
I can don't leave me  
  
There wasn't love the stars  
  
We're gonna be the good  
I can see you  
  
And I can stay  
  
I wanna leave it all the stars  
 

----- diversity: 0.5
----- Generating with seed: "ot just hear you  
Dumb dumb diddle, to "
ot just hear you  
Dumb dumb diddle, to your place  
  
I'm a light of my face  
I'm been to the sime  
  
I can't let's hard down  
And you got boy  
They  
And I want to go the something  
  
He's a back away to be a place  
And I'm love the countas  
  
Our end of the sun  
 

C:\Users\talvesdacosta\AppData\Local\Continuum\Anaconda2\envs\py35\lib\site-packages\ipykernel\__main__.py:4: RuntimeWarning: divide by zero encountered in log


s ro


In [26]:
generate_song(augment("I was a play of the streets"))


----- diversity: 0.2
----- Generating with seed: "             I was a play of the streets"
             I was a play of the streets  
I want to be the streets  
I'm a little love  
  
Well I'm a fantasy  
I think you wanna love the way  
  
I was still the way that I was the streets  
  
I think I can do  
I can't be  
  
I was a best all the right  
I'll be a better fine  
  
I think I can't the show  
  
I was stind of the way  
  
It's something the way that I don't want to be  
  
I was a little bit of my soul  
  
I am t

----- diversity: 0.5
----- Generating with seed: "             I was a play of the streets"
             I was a play of the streets  
We don't be me  
  
Sweet the light  
And I will really was a train  
I'll be of the stars that a)  
And I came away  
  
I'm the sound of my hold  
It's the shold of life  
I'm a man I see  
And you tell me to the way  
Come on fire I can't come to the stupiden  
It's all that I can't want to the sear of the way  
  
I push, it

In [28]:
model.save("generative_lstm.h5")

# Some generated songs

In [30]:
generate_song("My man you are crazy",temperature=0.5)


----- diversity: 0.5
----- Generating with seed: "                    My man you are crazy"
                    My man you are crazy  
At the lies, we the one of the day  
She was a man I know  
  
I am shine  
  
When it won't bay every home the streeps  
  
I've been so so completers  
The fingers I can do  
And I'm gonna be was on the steep  
  
There, she botted that in the world  
I'm gone  
They see my life  
The part my sweet brough  
  
[Chorus]  
(But I can be a good that I can  
  
Have me to your face  
  
The stree


##### Revisiting a classic by Witney Houston

In [31]:
generate_song("I will always love you",temperature=0.5)


----- diversity: 0.5
----- Generating with seed: "                  I will always love you"
                  I will always love you  
  
I can't tonge me is that all the one of your  
And I love you  
It was so left my heart  
  
The redde the head of me  
  
The things I be because they got to be  
  
Like them and something were the sky  
I she said with the most  
  
Chorus)  
  
If the world am was easy  
And I was when left a dimming a bad  
This got a brought my balled  
  
I love you  
I, the good that I was from a lit


##### And one by MIchael Jackson

In [33]:
generate_song("Don't stop till get enough",temperature=0.5)


----- diversity: 0.5
----- Generating with seed: "              Don't stop till get enough"
              Don't stop till get enough  
Christmas of nothing the the thing  
I'm a get all the way  
I love to the shot  
They was a little love  
The way only on  
And I'm not gonna make a strunger  
And the streets that you think you  
I am the shajof love  
I've know that you  
We don't live a come my fader  
I've got a way that I shake me  
  
You say they're lines, there's only  
You gotta help me  
  
Well, I'm living a little 
